In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random

## Set up. 
dataDir = '/gpfs/commons/groups/gursoy_lab/ajoglekar/Projects/2023_03_01_multiwayInteractions/2023_03_01_v0_dataGathering/v0_hypergraphSimulations/getMultiwayInteractions_fromBPChains/'
runDir = 'testWorkflow/iter1/'

In [3]:
with open(f'{dataDir}makeHyperGraphDict/10k_sample1/hyperEdges_3_600_750_final_chains.pkl','rb') as f:
    hpEdges = pickle.load(f)

numEdges = np.loadtxt(f'{dataDir}makeHyperGraphDict/10k_sample1/numEdges_3_600_750_final_chains.txt')

In [4]:
hpKeys = [k for k in hpEdges.keys()]
hpKeys_split = [k.split("_") for k in hpEdges.keys()]
keyCard = [len(item) for item in hpKeys_split]

In [6]:
len(hpKeys)

6809863

In [9]:
card = 3
ixList = [index for index,element in enumerate(keyCard) if element == card]
print("There are ",len(ixList),"reads")

readSupps = [hpEdges[hpKeys[ix]] for ix in ixList]
A = pd.Series(readSupps).describe()
A.to_pickle(f'{dataDir}{runDir}try.pkl')

There are  160827 reads


In [11]:
pd.read_pickle(f'{dataDir}{runDir}try.pkl')

count    160827.000000
mean        158.900253
std         718.025588
min           1.000000
25%           5.000000
50%          22.000000
75%          90.000000
max       30425.000000
dtype: float64

In [13]:
r = random.sample(range(10000),20)
# print(r)
print([hpKeys[i] for i in r])
# print([hpKeys_split[i] for i in r])

readsOfInterest = [hpKeys[i] for i in r]
with open(f'{dataDir}expectedReadCutoffEvaluation_v0/testSet/randomReads.pkl','wb') as f:
    pickle.dump(readsOfInterest,f)


['Bin45:49_Bin70:74_Bin95:99', 'Bin75:79_Bin190:194_Bin235:239_Bin245:249_Bin415:419', 'Bin250:254_Bin430:434', 'Bin85:89_Bin90:94_Bin140:144', 'Bin340:344_Bin350:354', 'Bin115:119_Bin125:129', 'Bin360:364_Bin365:369_Bin465:469_Bin470:474', 'Bin115:119_Bin155:159_Bin160:164_Bin165:169_Bin270:274_Bin310:314', 'Bin250:254_Bin255:259_Bin270:274_Bin275:279_Bin450:454', 'Bin40:44_Bin45:49_Bin85:89_Bin90:94_Bin140:144', 'Bin80:84_Bin150:154', 'Bin220:224_Bin240:244_Bin435:439', 'Bin5:9_Bin15:19_Bin20:24', 'Bin10:14_Bin110:114_Bin115:119_Bin120:124_Bin125:129', 'Bin250:254_Bin280:284_Bin285:289_Bin420:424', 'Bin160:164_Bin175:179', 'Bin15:19_Bin40:44_Bin80:84', 'Bin65:69_Bin365:369_Bin370:374', 'Bin20:24_Bin25:29_Bin45:49', 'Bin20:24_Bin95:99']


In [55]:
randomSplitKeyList = [hpKeys_split[i] for i in r]

indicesToConsider = []

for fileNum in range(1,10):
    print(fileNum)
    with open(f'{dataDir}chains_10k_500_projectionMtxOutput_sample2/binConcatInc_3_600_750_{fileNum}.pkl','rb') as f:
        A = pickle.load(f)
        for sk in randomSplitKeyList:
            print(sk)
            B = A.loc[sk]
            C = B.columns[B.sum() == B.shape[0]]
            if len(C) >0 : 
                print("Valid file")
                indicesToConsider.append(fileNum)
                break

1
['Bin10:14', 'Bin15:19', 'Bin275:279', 'Bin320:324']
Valid file
2
['Bin10:14', 'Bin15:19', 'Bin275:279', 'Bin320:324']
['Bin5:9', 'Bin10:14', 'Bin25:29']
['Bin15:19', 'Bin20:24', 'Bin25:29', 'Bin160:164']
['Bin10:14', 'Bin25:29', 'Bin30:34', 'Bin245:249']
['Bin0:4', 'Bin5:9', 'Bin35:39', 'Bin70:74', 'Bin145:149']
['Bin0:4', 'Bin65:69', 'Bin90:94', 'Bin95:99']
['Bin55:59', 'Bin75:79', 'Bin95:99', 'Bin105:109']
['Bin150:154', 'Bin320:324']
['Bin5:9', 'Bin35:39', 'Bin45:49', 'Bin240:244']
['Bin5:9', 'Bin35:39', 'Bin60:64', 'Bin70:74', 'Bin85:89', 'Bin150:154']
['Bin65:69', 'Bin70:74', 'Bin145:149']
['Bin0:4', 'Bin60:64', 'Bin75:79', 'Bin90:94']
['Bin115:119', 'Bin120:124', 'Bin125:129', 'Bin160:164']
['Bin0:4', 'Bin35:39', 'Bin55:59', 'Bin60:64']
Valid file
3
['Bin10:14', 'Bin15:19', 'Bin275:279', 'Bin320:324']
['Bin5:9', 'Bin10:14', 'Bin25:29']
Valid file
4
['Bin10:14', 'Bin15:19', 'Bin275:279', 'Bin320:324']
['Bin5:9', 'Bin10:14', 'Bin25:29']
['Bin15:19', 'Bin20:24', 'Bin25:29', 'Bin1

In [57]:
def getReads(inFile):
    """Read in a list of reads that we want to test our workflow on.
    Currently unsure if they are supposed to be random or interesting"""
    with open(inFile,'rb') as f:
        roi = pd.read_pickle(f)
        readsOfInterest = [r.split("_") for r in roi]
    return(readsOfInterest)

def checkIfValid(pickledFile,readsOfInterest):
    """need to check diff between pd.read_pickle and pickle.load"""
    for roi in readsOfInterest:
        print(roi)
        rois = pickledFile.loc[roi]
        nonZeroEdges = rois.columns[rois.sum() == rois.shape[0]]
        if len(nonZeroEdges) >0 : 
            print("Valid file")
            return True
        
def process_chunk(dataDir, inputDir, outDir, offDiagDist, chunk, chunk_size, readsOfInterest):
    """Process a chunk of files and write temporary output."""
    result_dict = {}
    numEdges = []

    start_file = (chunk - 1) * chunk_size + 1
    end_file = chunk * chunk_size

    for ix in range(start_file, end_file + 1):
        filePath = f'{dataDir}/{inputDir}/binConcatInc_3_600_750_{ix}.pkl'
        if os.path.isfile(filePath):
            c = False
            bIncDF = pd.read_pickle(filePath)
            c = checkIfValid(bIncDF,readsOfInterest)
            if c is True:
                result_dict = dfToDict(bIncDF, result_dict)
                nE = len(result_dict)
                numEdges.append(nE)

    # Write temporary output for the current chunk
    if result_dict:
        temp_output_file = f'{dataDir}/{outDir}hyperEdges_{offDiagDist}_600_750_chunk{chunk}_chains.pkl'
        with open(temp_output_file, 'wb') as f:
            pickle.dump(result_dict, f)

        temp_num_edges_file = f'{dataDir}{outDir}numEdges_{offDiagDist}_600_750_chunk{chunk}_chains.txt'
        np.savetxt(temp_num_edges_file, numEdges, delimiter='\t', fmt='%d')

    print(f"Chunk {chunk} done")

In [58]:
import os
readsOfInterest = getReads(f'{dataDir}expectedReadCutoffEvaluation_v0/testSet/randomReads.pkl')

process_chunk(dataDir, 'chains_10k_500_projectionMtxOutput_sample2/', 
              'expectedReadCutoffEvaluation_v0/testSet/', 3, 0, 5, readsOfInterest)

['Bin10:14', 'Bin15:19', 'Bin275:279', 'Bin320:324']
['Bin5:9', 'Bin10:14', 'Bin25:29']
Valid file


NameError: name 'dfToDict' is not defined

In [56]:
inputDir = 'chains_10k_500_projectionMtxOutput_sample2/'
ix = 3
filePath = f'{dataDir}/{inputDir}/binConcatInc_3_600_750_{ix}.pkl'